## Intro Enterprise deep research agent

Currently deep research tool doesn't support stream
* https://learn.microsoft.com/en-us/azure/ai-foundry/agents/how-to/tools/deep-research-samples
* https://learn.microsoft.com/en-us/azure/ai-foundry/agents/how-to/tools/deep-research

In [1]:
from typing import Any, Callable, Set, List, Optional
import re
import os
import time
import json
from azure.ai.projects import AIProjectClient
from azure.ai.agents import AgentsClient
from azure.ai.agents.telemetry import trace_function
from azure.ai.agents.models import (
    FunctionTool,
    RequiredFunctionToolCall,
    SubmitToolOutputsAction,
    ToolSet,
    ToolOutput,
    ThreadMessage,
    MessageRole,
    DeepResearchTool,
    ThreadRun,
)
import azure.ai.agents as agentslib
import azure.ai.projects as projectslib
from opentelemetry import trace
from azure.monitor.opentelemetry import configure_azure_monitor
from dotenv import load_dotenv, find_dotenv

# Your custom Python functions (for "fetch_datetime", etc.)
from utils.enterprise_functions import enterprise_fns

load_dotenv(dotenv_path=".env_westus", override=True)

from utils.fdyauth import AuthHelper
settings = AuthHelper.load_settings()
credential = AuthHelper.test_credential()

if credential:
    print('Environment and authentication OK')
else:
    print("please login first")

Environment and authentication OK


In [2]:
# new AI Foundry Project resource endpoint / old azure ai services endpoint from the hub/project
project_client = AIProjectClient(
    credential=credential,
    endpoint=settings.project_endpoint,
    # api_version=os.environ["PROJECT_API_VERSION"]
)
print("project_client api version:", project_client._config.api_version)
print(f"azure-ai-agents version: {agentslib.__version__}")
print(f"azure-ai-projects version: {projectslib.__version__}")

project_client api version: 2025-05-15-preview
azure-ai-agents version: 1.1.0b3
azure-ai-projects version: 1.0.0b12


In [3]:
try:
    bing_connection = project_client.connections.get(name=settings.bing_connection_name)
    # print(f"{bing_connection}")
    conn_id = bing_connection.id
    
    deep_research_tool = DeepResearchTool(
        bing_grounding_connection_id=conn_id,
        deep_research_model="o3-deep-research",
    )
    print("deep research > connected")
except Exception:
    bing_tool = None
    print("deep research failed > no connection found or permission issue")

deep research > connected


In [4]:
class LoggingToolSet(ToolSet):
    def execute_tool_calls(self, tool_calls: List[Any]) -> List[dict]:
        """
        Execute the upstream calls, printing only two lines per function:
        1) The function name + its input arguments
        2) The function name + its output result
        """

        # For each function call, print the input arguments
        for c in tool_calls:
            if hasattr(c, "function") and c.function:
                fn_name = c.function.name
                fn_args = c.function.arguments
                print(f"{fn_name} inputs > {fn_args} (id:{c.id})")

        # Execute the tool calls (superclass logic)
        raw_outputs = super().execute_tool_calls(tool_calls)

        # Print the output of each function call
        for item in raw_outputs:
            print(f"output > {item['output']}")

        return raw_outputs

# need an empty toolset to add tools
custom_functions = FunctionTool(enterprise_fns)

toolset = LoggingToolSet()

# if file_search_tool:
#      toolset.add(file_search_tool)
if deep_research_tool:
    toolset.add(deep_research_tool)
toolset.add(custom_functions)


for tool in toolset._tools:
    tool_name = tool.__class__.__name__
    print(f"tool > {tool_name}")
    for definition in tool.definitions:
        if hasattr(definition, "function"):
            fn = definition.function
            print(f"{fn.name} > {fn.description}")
        else:
            pass

tool > DeepResearchTool
tool > FunctionTool


In [5]:
# AGENT_NAME = "enterprise-deepresearch-agent"
AGENT_NAME = settings.agent_name
found_agent = None
all_agents_list = project_client.agents.list_agents()
for a in all_agents_list:
    if a.name == AGENT_NAME:
        found_agent = a
        break

model_name = settings.model_deployment_name

instructions = (
    "You are a helpful enterprise assistant at Contoso. "
    "You have access to following tools. \n\n"
    "## Tools:\n"
    " * deep_research: get information about company financial reportings\n"
)

project_client.agents.enable_auto_function_calls(tools=toolset, max_retry=4)

if found_agent:
    # print(found_agent)
    # Update the existing agent to use new tools
    agent = project_client.agents.update_agent(
        agent_id=found_agent.id,
        model=model_name,
        instructions=instructions,
        # tools=deep_research_tool.definitions,
        toolset=toolset

    )
    project_client.agents.enable_auto_function_calls(tools=toolset) 
    print(f"reusing agent > {agent.name} (id: {agent.id})")
else:
    agent = project_client.agents.create_agent(
        model=model_name,
        name=AGENT_NAME,
        instructions=instructions,
        # tools=deep_research_tool.definitions
        toolset=toolset,
    )
    print(f"creating agent > {agent.name} (id: {agent.id})")

reusing agent > enterprise-research-agent (id: asst_7lKJLNKtsmqUIhjlM6Zx4sDB)


In [6]:
thread = project_client.agents.threads.create()
print(f"Created thread, ID: {thread.id}")

Created thread, ID: thread_zRN6TUHnCh5Ue7jii3p9XONd


In [7]:
# Create message to thread
message = project_client.agents.messages.create(
    thread_id=thread.id,
    role=MessageRole.USER,
    content=(
        "Give me the latest research into quantum computing over the last year."
    ),
)
print(f"Created message, ID: {message.id}")

Created message, ID: msg_8x0fOTkbXrTdZjv6Zgc0RjW4


In [8]:
root_dir = os.path.abspath(os.getcwd())
print(root_dir)

c:\Users\yingdingwang\Documents\VCS\democollections\ai-foundry-workshop\01-foundamentals


In [9]:
def fetch_and_print_new_agent_response(
    thread_id: str,
    agents_client: AgentsClient,
    last_message_id: Optional[str] = None,
) -> Optional[str]:
    response = agents_client.messages.get_last_message_by_role(
        thread_id=thread_id,
        role=MessageRole.AGENT,
    )
    if not response or response.id == last_message_id:
        return last_message_id  # No new content

    print("\nAgent response:")
    print("\n".join(t.text.value for t in response.text_messages))

    for ann in response.url_citation_annotations:
        print(f"URL Citation: [{ann.url_citation.title}]({ann.url_citation.url})")

    return response.id

def create_research_summary(
        message : ThreadMessage,
        subdir="data", filename: str = "research_summary.md"
) -> None:
    if not message:
        print("No message content provided, cannot create research summary.")
        return

    root_dir = os.path.abspath(os.getcwd())
    filepath = os.path.join(root_dir, subdir, filename)
    print(filepath)

    with open(filepath, "w", encoding="utf-8") as fp:
        # Write text summary
        text_summary = "\n\n".join([t.text.value.strip() for t in message.text_messages])
        fp.write(text_summary)

        # Write unique URL citations, if present
        if message.url_citation_annotations:
            fp.write("\n\n## References\n")
            seen_urls = set()
            for ann in message.url_citation_annotations:
                url = ann.url_citation.url
                title = ann.url_citation.title or url
                if url not in seen_urls:
                    fp.write(f"- [{title}]({url})\n")
                    seen_urls.add(url)

    print(f"Research summary written to '{filepath}'.")

In [10]:
def run_agent(thread_id: str, agent_id: str, show_output: bool = True)-> ThreadRun:
    print(f"Start processing the message... this may take a few minutes to finish. Be patient!")
    # Poll the run as long as run status is queued or in progress
    run = project_client.agents.runs.create(thread_id=thread.id, agent_id=agent.id)
    last_message_id = None
    print(f"Run started with ID: {run.id}, status: {run.status}")
    while run.status in ("queued", "in_progress"):
        time.sleep(1)
        run = project_client.agents.runs.get(thread_id=thread.id, run_id=run.id)
        
        if show_output:
            last_message_id = fetch_and_print_new_agent_response(
                thread_id=thread.id,
                agents_client=project_client.agents,
                last_message_id=last_message_id,
            )
        print(f"Run status: {run.status}")

    print(f"Run finished with status: {run.status}, ID: {run.id}")

    if run.status == "failed":
        print(f"Run failed: {run.last_error}")
    return run

In [11]:
run = run_agent(thread_id=thread.id, agent_id=agent.id)

Start processing the message... this may take a few minutes to finish. Be patient!
Run started with ID: run_LMrwOcPr4kCPuHkxkpdRQD6r, status: RunStatus.QUEUED

Agent response:
Could you please specify any particular areas or aspects of quantum computing you are most interested in? For example, are you looking for breakthroughs in quantum hardware, algorithms, error correction, applications, or commercial developments? Also, would you like the research focus to be academic papers, industry advancements, or both? This will help me tailor the research to your needs.
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.COMPLETED
Run finished with status: RunStatus.COMPLETED, ID: run_LMrwOcPr4kCPuHkxkpdRQD6r


In [12]:
message = project_client.agents.messages.create(
    thread_id=thread.id,
    role=MessageRole.USER,
    content=(
        # "Recent advancements in hardward, algorithms, applications. I prefer a mix of both academic publication and industry news in a structured report. Research and news to the past year."
        "Recent advancements in hardward. I prefer only industry news in a structured report. Research and news to the past 3 month."
    ),
)
print(f"Created message, ID: {message.id}")

Created message, ID: msg_ve4gtZ7SLHty32Pq4hQtvG2e


In [13]:
run = run_agent(thread_id=thread.id, agent_id=agent.id, show_output=False)

Start processing the message... this may take a few minutes to finish. Be patient!
Run started with ID: run_gzHTT4sFfTEgFtsQKu1Q9h6h, status: RunStatus.QUEUED
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_

In [14]:
# Fetch the final message from the agent in the thread and create a research summary
final_message = project_client.agents.messages.get_last_message_by_role(
    thread_id=thread.id, role=MessageRole.AGENT
)
if final_message:
    create_research_summary(final_message, subdir="data", filename="research_summary.md")

c:\Users\yingdingwang\Documents\VCS\democollections\ai-foundry-workshop\01-foundamentals\data\research_summary.md
Research summary written to 'c:\Users\yingdingwang\Documents\VCS\democollections\ai-foundry-workshop\01-foundamentals\data\research_summary.md'.


In [15]:
last_run = project_client.agents.runs.get(thread_id=thread.id, run_id=run.id)

In [16]:
## Load the report
report_path = os.path.join(os.getcwd(), "data", "research_summary.md")
if os.path.exists(report_path):
    with open(report_path, "r", encoding="utf-8") as file:
        report_content = file.read()
    print("Research Summary Report:")
    print(report_content)

Research Summary Report:
Final Report:
# Advancements in Quantum Computing Hardware (April–July 2025)

Quantum computing hardware has seen significant progress in the past three months. Major industry players have announced new processors, improved performance metrics, and roadmap milestones toward large-scale quantum systems. Below is a structured overview of the latest developments, organized by company and technology, with sources from recent news and official releases.

## IBM: Roadmap to Fault Tolerance and New Quantum Systems

- **Fault-Tolerant Quantum Roadmap:** In June 2025, IBM unveiled an updated quantum roadmap aiming to build the first large-scale, fault-tolerant quantum computer by 2029【33:4†source】. Codenamed **IBM Quantum Starling**, this future machine is expected to perform **20,000× more operations** than today’s quantum processors【33:4†source】. IBM’s plan introduces intermediate processors – *Loon* (expected 2025), *Kookaburra* (2026), and *Cockatoo* (2027) – which 